In [5]:
#TRain a transformer

# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt 


--2025-01-21 21:35:01--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  3.20MB/s    in 0.3s    

2025-01-21 21:35:01 (3.20 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()


In [3]:
print(f"length of the dataset in characters: {len(text)}")

length of the dataset in characters: 1115394


In [4]:
##Sort the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
all_chars = ''.join(chars)

print(f"{all_chars}")
print(f"{vocab_size} chars")
      


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65 chars


Tokenizer

In [5]:
#Lets work on tokenization

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decoder = lambda l: [itos[c] for c in l]



In [ ]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print("Every integer in data represents a single character ", data.shape)


Every integer in data represents a single character  torch.Size([1115394])


In [7]:
#split
n = int(0.9*len(data))

train_data = data[:n]
val_data = data[n:]

In [8]:
context_length = 8
train_data[:context_length+1]

"8 examples come out of nine characters given the context for interpreting the next prediction "

'8 examples come out of nine characters given the context for interpreting the next prediction '

In [9]:
x = train_data[:context_length]
y = train_data[1:context_length+1]

for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4
context_length = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,)) #tensor of four integers representing four characters from the text
    # So for the random characters, we get i and then up to the block size. Then we put in a bigger tensor
    x = torch.stack([data[i:i+context_length] for i in ix])
    #same thing for y
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    
    return x,y

xb, yb = get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

#This is where we show all of the 'examples' within our dataset. This is helpful for incorporating different context lengths
for b in range(batch_size):
    for t in range(context_length):
        context = xb[b, :t+1]
        target = yb[b,t]
        
        print(f"when input is {context.tolist()} the target: {target}")



inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        
        #Create embedding table where each index(representing individual characters) 
        # will pull out a row from the embedding table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
        
    def forward(self, idx, targets = None):
       
        #idx and targets are both (b, context_length) tensor of integers
        logits = self.token_embedding_table(idx) #output is (B,T,C)
        
        if targets is None: 
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) #-> for cross entropy
            targets = targets.view(B*T) #reshaping
            
            #negative log likelihood
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens):
        
        #idx is (B,context_length) array of indices in the current context. Predicting token by token
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            
            #focus only on the last example (where we consider the max token length TODO: Understand why here)
            #These are the predictions for what comes next
            logits = logits[:,-1,:]
            
            #Apply softmax to get the probabilities
            probs = F.softmax(logits, dim = -1) #B, C
            
            idx_next = torch.multinomial(probs, num_samples = 1) #for each context length in the batch return the next selection based on probabilities distribution
            
            idx = torch.cat((idx, idx_next), dim = 1) #concatenate to the idx to create (B, T+1)
            #print(f"current status of idx: {idx}")
            
        return idx
    

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

starting_index = torch.zeros((1,1), dtype = torch.long) #Kicking off generation with zero tensors
sample_generated = m.generate(starting_index, max_new_tokens=100)
print(f"sample generated: {sample_generated.shape}")
#Sample generated works on the order of batches so we need to grab the return batch (it's effectively nested in a list)
batch_of_interest = sample_generated[0]

#Turn it into a list and decode
decoded = decoder(batch_of_interest.tolist())
"""(batch size, context_length (really number fo tokens we are looking at, 
embedding length per token))"""

print(f"Decoded output: {''.join(decoded)}")

sample generated: torch.Size([1, 101])
Decoded output: 
Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [ ]:
#Create optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [39]:


batch_size = 32
for steps in range(1000):
    
    #get batch
    xb, yb = get_batch('train')
    
    #forward pass
    logits, loss = m(xb, yb)
    
    #zeroing the gradients
    optimizer.zero_grad(set_to_none = True)
    #getting gradients for parameters
    loss.backward()
    #upgrade parameters using optimizer
    optimizer.step()
    
print(loss.item())
    


2.6088967323303223


In [41]:
starting_index = torch.zeros((1,1), dtype = torch.long) #Kicking off generation with zero tensors
sample_generated = m.generate(starting_index, max_new_tokens=300)
print(f"sample generated: {sample_generated.shape}")
#Sample generated works on the order of batches so we need to grab the return batch (it's effectively nested in a list)
batch_of_interest = sample_generated[0]

#Turn it into a list and decode
decoded = decoder(batch_of_interest.tolist())
"""(batch size, context_length (really number fo tokens we are looking at, 
embedding length per token))"""

print(f"Decoded output: {''.join(decoded)}")

sample generated: torch.Size([1, 301])
Decoded output: 
AchinkOvin Hooly zolIn kthitollvkefa; 'vezIO$ARO:
HO:
By tu
PQ-d?e n ick:isanfeConus: tas, bycon,
Eli!

IHALETETEOUy-l, sthe.
CONEDuse Mlitiard tf'd EnQhOMyarerytrevNTa!
atD: tanTo sis; hitRWhedwid.
puDYonymy thandoTowQ!
ve:$ys s clllcUzPUpithinEneshh f r Isptlve II'A:'I!
pJOFtomukeg.
G wf tYZxrolb



#### Self-attention

In [43]:
torch.manual_seed(1337)
B,T,C = 4,8,2

x = torch.randn(B,T,C)
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [67]:
#toy example with loops

torch.manual_seed(42)
B,T,C = 4,8,2

x = torch.randn(B,T,C)
x.shape
#Tokens should not look ahead at future tokens. 5th token should only look backward. 

xbow = torch.zeros((B,T,C))
xbow2 = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev, 0)
        
        xprev2 = x[b][:t+1]
        xbow2[b][t] = torch.mean(xprev, 0)
        
#print("Xbow: ",xbow)
#print("Xbow2: ", xbow2)

In [69]:
##another toy example with matrix multiplication
#torch.tril(torch.ones(3,3))
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a /torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()

c = a@b
print('a= ')
print(a)

print('--')
print("b=")
print(b)

print('--')
print('c=')
print(c)

"""Think about what a/torch.sum(a,1, keepdim = True) means here. We're normalizing each row of a to 1 
With this structure we have a means of calculating the 'average' input each character gives to the remaining characters
in the row. However, be careful! Averages are not the only way to calculate this!"""

a= 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


"Think about what a/torch.sum(a,1, keepdim = True) means here. We're normalizing each row of a to 1 \nWith this structure we have a means of calculating the 'average' input each character gives to the remaining characters\nin the row. However, be careful! Averages are not the only way to calculate this!"

## Multiplication method

In [ ]:
triangular = torch.tril(torch.ones(T,T))
triangular = triangular/torch.sum(triangular, 1, keepdim = True)
print("xmatrix, ", x.shape)
print("triangular: ",triangular.shape)
c = triangular@x # ---> (T,T) @ (B,T,C) --> (B,T,T) @ *(B,T,C)

"""Think about what we demonstrated here! We calculated the averages of the triangular which represent weights from preceding tokens"""

xmatrix,  torch.Size([4, 8, 2])
triangular:  torch.Size([8, 8])


In [71]:
torch.allclose(c,xbow)

True

In [59]:
triangular

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [53]:
x_matrix

tensor([[[ 1.6318e+00, -6.3176e-01],
         [ 2.8210e-01,  7.1790e-01],
         [ 9.6083e-01,  3.9169e-02],
         [ 9.3684e-01,  6.3157e-02],
         [ 2.3620e-01,  7.6380e-01],
         [ 7.2572e-01,  2.7428e-01],
         [-6.1758e+01,  6.2758e+01],
         [-1.4805e+00,  2.4805e+00]],

        [[ 1.1155e+00, -1.1548e-01],
         [ 2.2847e-01,  7.7153e-01],
         [ 1.3194e+00, -3.1939e-01],
         [ 7.1558e-01,  2.8442e-01],
         [-8.7724e-02,  1.0877e+00],
         [ 7.7602e-01,  2.2398e-01],
         [-1.2653e+00,  2.2653e+00],
         [ 3.7786e-01,  6.2214e-01]],

        [[ 7.2519e-01,  2.7481e-01],
         [ 8.9260e-01,  1.0740e-01],
         [ 1.2256e-01,  8.7744e-01],
         [ 7.9453e-01,  2.0547e-01],
         [ 3.5456e-01,  6.4544e-01],
         [-5.7665e-01,  1.5767e+00],
         [ 1.9021e+00, -9.0207e-01],
         [ 4.6270e-01,  5.3730e-01]],

        [[ 1.9531e+00, -9.5306e-01],
         [ 1.2566e+00, -2.5655e-01],
         [-2.5438e+00,  3.5438e+

In [ ]:
##Version 3: 

tril = torch.tril(torch.ones(T,T))

wei = torch.zeros((T,T)) #--> really affiniities that are dependent upon data and learning. We learn these weights!
wei = wei.masked_fill(tril == 0, float('-inf')) #future cannot communicate with the past
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
#Version 4 Self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 #batch, time, channels

#Lets see a single head perform self-atttention
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size,  bias = False)
value = nn.Linear(C, head_size, bias = False)

k = key(x) #(B,T, 16)
q = query(x) #(B,T,16)

wei = q@k.transpose(-2, -1) * (head_size**0.5) #transpose last two dimensions -> B,T,16 @ (B,16,T) ---> B, T, T. Showing affinities between tokens in queries and keys!

#^note the scaling above

x = torch.randn(B,T,C)
tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))

wei = wei.masked_fill(tril == 0, float('-inf')) #prevent future masking!
wei = F.softmax(wei, dim = -1)#nice aggregation so weights are useful!

v = value(x)
out = wei@v #v is the vectors that we use to aggregate the impact of Q and K on tokens projected to V subspace
out.shape

torch.Size([4, 8, 16])

In [81]:
x[0].shape

torch.Size([8, 32])

In [82]:
v[0]

tensor([[ 0.0512,  0.0713,  0.2697, -0.3172,  0.9616,  1.0172, -0.3177, -0.2535,
          0.2846, -0.1037,  0.4592,  0.6528,  0.4756, -0.3411,  0.0508, -0.7247],
        [-0.2612,  0.3187, -0.2735, -0.7162,  0.6417,  0.5328,  0.1999, -0.8511,
          0.8913,  0.0559,  0.8147,  0.1956,  0.1320,  0.1563, -0.7694, -0.1499],
        [-0.1229, -0.1420, -0.4644, -0.4878, -0.9135, -0.8398,  0.9232,  0.8595,
         -0.0495, -0.4269, -0.5219, -0.3808,  0.5760,  0.1216,  0.0963,  0.9161],
        [-0.2617, -0.6189,  0.0530,  0.8612, -0.5344,  0.6039,  0.1786,  0.0475,
          0.3528,  0.1594,  0.0934,  0.3796,  0.4497,  0.6299, -0.1479,  0.6181],
        [ 0.5290,  0.2105, -0.7123, -0.4573,  0.3816,  0.0186, -0.0249,  0.3256,
         -0.3851,  0.6592, -0.7780, -0.7986,  0.1654, -0.7854,  0.3081, -0.0177],
        [ 0.0422,  0.2055,  0.7199, -0.0295, -0.0580, -0.3567,  0.3860, -0.4306,
          0.2643,  0.0803, -0.0359,  0.6603, -0.1632,  0.0101, -0.0192,  0.7446],
        [ 0.0993,  0.1

In [75]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
"Query - what am I looking for"
"Key - what do I contain"
"Value - we use this aggergate the token impacts for a single head. Here's what I will communicate to you projecting token embeddings to head space"
"affinity - dot product of Q@K"